In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

import sys
sys.path.append('/Users/hzm/Desktop/YangTeng/code/venv/code/00.module')
import Proxy
import UA
import time

# = = = = = = = = = = = = = = = = = =

import pandas as pd
from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_selenium())
option.add_argument('user-agent=' + UA.get_UA())
option.page_load_strategy = 'eager'

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu.xlsx', header=0)
list_url = df_menu['Url'].to_list()

print('总数量：' + str(len(list_url)))
print()

df = pd.DataFrame()

# = = = = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)
                browser.maximize_window()

                # = = = = = = = = = = = = = = = = = =

                browser.get(url)
                
                time.sleep(3)
                
                # = = = = = = = = = = = = = = = = = =
                
                if len(browser.find_elements(by=By.XPATH, value='//span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]')) != 0:
                    break
                    
                # = = = = = = = = = = = = = = = = = =

                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//tbody[contains(@class, "listing-inner") and contains(@class, "altrow-a-1")]')))
                list_part = browser.find_elements(by=By.XPATH, value='//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = = = = =

                list_part_number = [part.find_element(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]') for part in list_part]

                # = = = = = = = = = = = = = = = = = =
                
                part_index = [part_number.text for part_number in list_part_number].index(url.split(',')[-3])
                
                # = = = = = = = = = = = = = = = = = =
                
                manufacturer = list_part[part_index].find_element(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]').text

                # = = = = = = = = = = = = = = = = = =
                
                list_part_number[part_index].click()
                time.sleep(7)
                list_vehicle = browser.find_element(by=By.XPATH, value='//div[contains(@id, "buyersguidepopup-outer_c")]').find_elements(by=By.XPATH, value='./descendant::table[contains(@class, "nobmp")]')
                if len(list_vehicle) == 0:
                    vehicle = ''
                else:
                    vehicle = list_vehicle[0].text

                # = = = = = = = = = = = = = = = = = =

                note_1 = ';'.join(row.text for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]'))

                # = = = = = = = = = = = = = = = = = =

                info = ';'.join(row.get_attribute('href') for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/a[contains(@class, "ra-btn") and contains(@class, "ra-btn-moreinfo")]'))

                # = = = = = = = = = = = = = = = = = =
                
                note_2 = ';'.join(row.text for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row") and not(contains(@class, "moreinfo-truck"))]'))

                # = = = = = = = = = = = = = = = = = =

                oe = ';'.join(row.get_attribute('textContent') for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::span[contains(@title, "Replaces these Alternate/ OE Part Numbers")]'))
                oe = ';'.join(oe.split(', '))
                
                # = = = = = = = = = = = = = = = = = =

                src = ';'.join(row.get_attribute('src') for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::img[contains(@id, "inlineimg_thumb")]'))

                # = = = = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Manufacturer': manufacturer,
                                         'Vehicle': vehicle,
                                         'Alternate_OE_Part_Numbers': oe,
                                         'Note_1': note_1,
                                         'Note_2': note_2,
                                         'Pic': '',
                                         'Url': url,
                                         'Info': info,
                                         'Src': src}])
                
                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url.split(',')[-3] +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(25):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('输出中...')
df.to_excel('./part.xlsx', index=False)
print()
print('搞定！')

总数量：511

73005  <->  [ok] - 剩余数量：486
73001  <->  [ok] - 剩余数量：485
73011  <->  [ok] - 剩余数量：484
73002  <->  [ok] - 剩余数量：483
73013  <->  [ok] - 剩余数量：482
73010  <->  [ok] - 剩余数量：481
37535  <->  [ok] - 剩余数量：480
37534  <->  [ok] - 剩余数量：479
73009  <->  [ok] - 剩余数量：478
37539  <->  [ok] - 剩余数量：477
73012  <->  [ok] - 剩余数量：476
37537  <->  [ok] - 剩余数量：475
37530  <->  [ok] - 剩余数量：474
37536  <->  [ok] - 剩余数量：473
73007  <->  [ok] - 剩余数量：472
37538  <->  [ok] - 剩余数量：471
37533  <->  [ok] - 剩余数量：470
37531  <->  [ok] - 剩余数量：469
73000  <->  [ok] - 剩余数量：468
73006  <->  [ok] - 剩余数量：467
37540  <->  [ok] - 剩余数量：466
73004  <->  [ok] - 剩余数量：465
73003  <->  [ok] - 剩余数量：464
73016  <->  [ok] - 剩余数量：463
73008  <->  [ok] - 剩余数量：462
73015  <->  [ok] - 剩余数量：461
37532  <->  [ok] - 剩余数量：460
73014  <->  [ok] - 剩余数量：459
73030  <->  [ok] - 剩余数量：458
73033  <->  [ok] - 剩余数量：457
73018  <->  [ok] - 剩余数量：456
73017  <->  [ok] - 剩余数量：455
73019  <->  [ok] - 剩余数量：454
73021  <->  [ok] - 剩余数量：453
73025  <->  [ok] - 剩余数量：452
73020  <-> 

73348  <->  [ok] - 剩余数量：193
73332  <->  [ok] - 剩余数量：192
73349  <->  [ok] - 剩余数量：191
73351  <->  [ok] - 剩余数量：190
73357  <->  [ok] - 剩余数量：189
73352  <->  [ok] - 剩余数量：188
73355  <->  [ok] - 剩余数量：187
73353  <->  [ok] - 剩余数量：186
73356  <->  [ok] - 剩余数量：185
73361  <->  [ok] - 剩余数量：184
73364  <->  [ok] - 剩余数量：183
73360  <->  [ok] - 剩余数量：182
73366  <->  [ok] - 剩余数量：181
73354  <->  [ok] - 剩余数量：180
73363  <->  [ok] - 剩余数量：179
73370  <->  [ok] - 剩余数量：178
73368  <->  [ok] - 剩余数量：177
73376  <->  [ok] - 剩余数量：176
73369  <->  [ok] - 剩余数量：175
73365  <->  [ok] - 剩余数量：174
73367  <->  [ok] - 剩余数量：173
73372  <->  [ok] - 剩余数量：172
73359  <->  [ok] - 剩余数量：171
73374  <->  [ok] - 剩余数量：170
73362  <->  [ok] - 剩余数量：169
73377  <->  [ok] - 剩余数量：168
73371  <->  [ok] - 剩余数量：167
73379  <->  [ok] - 剩余数量：166
73380  <->  [ok] - 剩余数量：165
73381  <->  [ok] - 剩余数量：164
73383  <->  [ok] - 剩余数量：163
73373  <->  [ok] - 剩余数量：162
73129  <->  [ok] - 剩余数量：161
73384  <->  [ok] - 剩余数量：160
73386  <->  [ok] - 剩余数量：159
73385  <->  [ok] - 剩